# Web Crawling Part 2.

## beautifulsoup
- 웹 페이지를 구문 분석하고 데이터를 추출하는 데 도움이 되는 라이브러리
- 웹 페이지의 HTML 및 XML 파일을 쉽게 탐색하고, 검색하고, 수정할 수 있게 해줌
- 다양한 파서(구문 분석기)를 사용하여 문서를 구문 분석하고, 태그를 찾거나 데이터를 추출하는 것이 가능
- BeautifulSoup은 웹 스크래핑과 데이터 마이닝에서 매우 유용하게 사용

In [36]:
import requests
from bs4 import BeautifulSoup

### html 문자열 파싱
- 문자열로 정의된 html 데이터 파싱하기

In [37]:
html = '''
<html>
  <head>
    <title>BeautifulSoup test</title>
  </head>
  <body>
    <div id='upper' class='test' custom='good'>
      <h3 title='Good Content Title'>Contents Title</h3>
      <p>Test contents</p>
    </div>
    <div id='lower' class='test' custom='nice'>
      <p>Test Test Test 1</p>
      <p>Test Test Test 2</p>
      <p>Test Test Test 3</p>
    </div>
  </body>
</html>'''

In [38]:
html_file = open('html_file.html', 'w')
html_file.write(html)
html_file.close()

## find 함수
- 특정 html tag를 검색
- 검색 조건을 명시하여 찾고자하는 tag를 검색
- 문서의 첫 번째 태그나 조건에 맞는 요소를 반환
- 만약 일치하는 요소가 없다면 None을 반환

In [39]:
soup = BeautifulSoup(html)

In [40]:
soup.find('h3')

<h3 title="Good Content Title">Contents Title</h3>

In [41]:
soup.find('p')

<p>Test contents</p>

In [42]:
soup.find('div', id='lower')

<div class="test" custom="nice" id="lower">
<p>Test Test Test 1</p>
<p>Test Test Test 2</p>
<p>Test Test Test 3</p>
</div>

In [43]:
attrs = {'id': 'upper', 'class': 'test'}
soup.find('div', attrs=attrs)

<div class="test" custom="good" id="upper">
<h3 title="Good Content Title">Contents Title</h3>
<p>Test contents</p>
</div>

## find_all 함수
- HTML 또는 XML 문서 내에서 지정된 조건에 일치하는 모든 요소를 찾는 데 사용
- 이 함수는 일치하는 모든 요소의 리스트를 반환
- 만약 일치하는 요소가 없다면 빈 리스트를 반환
- find가 조건에 만족하는 하나의 tag만 검색한다면, find_all은 조건에 맞는 모든 tag를 리스트로 반환

In [44]:
soup.find_all('div', class_='test')

[<div class="test" custom="good" id="upper">
 <h3 title="Good Content Title">Contents Title</h3>
 <p>Test contents</p>
 </div>,
 <div class="test" custom="nice" id="lower">
 <p>Test Test Test 1</p>
 <p>Test Test Test 2</p>
 <p>Test Test Test 3</p>
 </div>]

## get_text 함수
- tag안의 value를 추출
- 부모 tag의 경우, 모든 자식 tag의 value를 추출

In [46]:
tag = soup.find('h3')
print(tag)
print(tag.get_text())

<h3 title="Good Content Title">Contents Title</h3>
Contents Title


In [47]:
tag = soup.find('p')
print(tag)
print(tag.get_text())

<p>Test contents</p>
Test contents


In [48]:
tag = soup.find('div', id='upper')
print(tag)
print(tag.get_text().strip())

<div class="test" custom="good" id="upper">
<h3 title="Good Content Title">Contents Title</h3>
<p>Test contents</p>
</div>
Contents Title
Test contents


## attribute 값 추출하기
- 경우에 따라 추출하고자 하는 값이 attribute에도 존재함
- 이 경우에는 검색한 tag에 attribute 이름을 [ ]연산을 통해 추출가능
- 예) div.find('h3')['title']

In [49]:
tag = soup.find('h3')
print(tag)
print(tag['title'])

<h3 title="Good Content Title">Contents Title</h3>
Good Content Title


## 키워드 기반 데이터 추출해보기
 - tag를 추출할때는 가장 그 tag를 쉽게 특정할 수 있는 속성을 사용
  - id의 경우 원칙적으로 한 html 문서 내에서 유일

In [71]:
import requests
from bs4 import BeautifulSoup

In [72]:
keyword = '비트코인'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}'

In [73]:
# 웹 페이지 접속
response = requests.get(url)

In [74]:
# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# 불러온 HTML 코드(soup)에서 뉴스 제목이 포함된 부분만 articles에 저장
# 제목 링크는 공통적으로 data-heatmap-target=".tit" 속성을 갖고 있는 <a> 태그에 존재
articles = soup.select('a[data-heatmap-target=".tit"]')
articles

In [76]:
article = articles[0]
title = article.text
title

'압수된 비트코인 9조…英서 자금세탁 중국인 징역 11년8개월'

In [78]:
import pandas as pd

title_tot = pd.DataFrame([article.text for article in articles], columns=['title'])
title_tot.head()

,title
0,압수된 비트코인 9조…英서 자금세탁 중국인 징역 11년8개월
1,압수 비트코인만 9조…13만명에 사기 친 중국인의 최후
2,"'비트코인 9조 세탁' 중국 여성, 英 법원서 징역 11년 8개월형"
3,"'9조 비트코인 사기' 40대 여성, 월세 3000만원 집 살더니 결국"
4,"中 다단계 사기범, 英서 징역 11년8개월…비트코인 9조 압수"
